In [ ]:
# Load cleaned order data
orders_df = spark.read.option("header", True).option("inferSchema", True).csv("/FileStore/tables/orders_cleaned.csv")
orders_df.createOrReplaceTempView("orders")

In [ ]:
# Load latest delivery status
status_df = spark.read.option("header", True).option("inferSchema", True).csv("/FileStore/tables/delivery_status.csv")
status_df.createOrReplaceTempView("delivery_status")

In [ ]:
# Join on order_id and get updated status
orders_updated_df = orders_df.alias("o").join(
    status_df.select("order_id", "status", "last_updated").alias("s"),
    on="order_id",
    how="left"
).drop("o.status").withColumnRenamed("s.status", "status")

orders_updated_df.createOrReplaceTempView("orders_updated")

In [ ]:
orders_updated_df.write.mode("overwrite").format("delta").save("/mnt/datalake/processed/orders_updated_delta")
orders_updated_df.write.mode("overwrite").option("header", True).csv("/FileStore/output/orders_updated_csv")